In [5]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd


In [3]:
model = load_model('model.h5')

In [12]:
with open("label_encoding.pickle",'rb') as file:
    label_encode = pickle.load(file)

with open("one_hot_encoding.pickle",'rb') as file:
    one_hot_encode = pickle.load(file)

with open("Standard_Scaler.pickle",'rb') as file:
    standard_scaler = pickle.load(file)

In [7]:
data = pd.DataFrame([
    [720, "France", "Male", 35, 5, 65000.00, 2, 1, 1, 52000.00],
    [610, "Spain", "Female", 45, 3, 120000.00, 1, 0, 0, 95000.00],
    [480, "Germany", "Male", 52, 1, 150000.00, 3, 1, 0, 110000.00],
    [680, "France", "Female", 28, 2, 30000.00, 1, 1, 1, 45000.00],
    [590, "Spain", "Male", 40, 8, 0.00, 2, 1, 0, 78000.00]
], columns=[
    "CreditScore", "Geography", "Gender", "Age", "Tenure", "Balance",
    "NumOfProducts", "HasCrCard", "IsActiveMember", "EstimatedSalary"
])

In [13]:

data['Gender'] = label_encode.transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,720,France,1,35,5,65000.0,2,1,1,52000.0
1,610,Spain,0,45,3,120000.0,1,0,0,95000.0
2,480,Germany,1,52,1,150000.0,3,1,0,110000.0
3,680,France,0,28,2,30000.0,1,1,1,45000.0
4,590,Spain,1,40,8,0.0,2,1,0,78000.0


In [14]:

encoded_df = one_hot_encode.fit_transform(data[['Geography']])
encoded_df = encoded_df.toarray()
features = one_hot_encode.get_feature_names_out(['Geography'])

In [15]:
geo_df = pd.DataFrame(encoded_df,columns=features)
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [16]:
data = data.drop(['Geography'],axis = 1)
data = pd.concat([data,geo_df],axis = 1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,720,1,35,5,65000.0,2,1,1,52000.0,1.0,0.0,0.0
1,610,0,45,3,120000.0,1,0,0,95000.0,0.0,0.0,1.0
2,480,1,52,1,150000.0,3,1,0,110000.0,0.0,1.0,0.0
3,680,0,28,2,30000.0,1,1,1,45000.0,1.0,0.0,0.0
4,590,1,40,8,0.0,2,1,0,78000.0,0.0,0.0,1.0


In [18]:
scaled_data = standard_scaler.transform(data)
scaled_data

c:\Users\kilar\OneDrive\Desktop\Gen AI\venv\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


array([[ 7.20000000e+02,  1.00000000e+00,  3.50000000e+01,
         5.00000000e+00,  6.50000000e+04,  2.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  5.20000000e+04,
         1.00000000e+00, -1.68753900e-17,  1.59872116e-17],
       [ 6.10000000e+02, -1.24344979e-17,  4.50000000e+01,
         3.00000000e+00,  1.20000000e+05,  1.00000000e+00,
         9.68114477e-17,  6.21724894e-17,  9.50000000e+04,
        -9.76996262e-17, -1.68753900e-17,  1.00000000e+00],
       [ 4.80000000e+02,  1.00000000e+00,  5.20000000e+01,
         1.00000000e+00,  1.50000000e+05,  3.00000000e+00,
         1.00000000e+00,  6.21724894e-17,  1.10000000e+05,
        -9.76996262e-17,  1.00000000e+00,  1.59872116e-17],
       [ 6.80000000e+02, -1.24344979e-17,  2.80000000e+01,
         2.00000000e+00,  3.00000000e+04,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  4.50000000e+04,
         1.00000000e+00, -1.68753900e-17,  1.59872116e-17],
       [ 5.90000000e+02,  1.00000000e+00,  4.0000000

In [20]:
pred_prob = model.predict(scaled_data)

1/1 [==============================] - 1s 924ms/step


In [21]:
pred_prob

array([[1.],
       [1.],
       [1.],
       [1.],
       [0.]], dtype=float32)

In [22]:
for i in range(0,len(pred_prob)):
    if pred_prob[i][0] > 0.5:
        print("Customer is likely to churn")
    else:
        print("Customer is unlikely to churn")

Customer is likely to churn
Customer is likely to churn
Customer is likely to churn
Customer is likely to churn
Customer is unlikely to churn
